In [1]:
import numpy as np
import tensorflow as tf
from PIL import Image
import time

In [2]:
def load_model(model_name):
    model_dir = ".\\models\\"+model_name+"\\saved_model"
    model = tf.saved_model.load(str(model_dir))
    model = model.signatures['serving_default']
    return model

In [3]:
model_name = 'ssd_mobilenet_v2_coco_2018_03_29'
model_name = 'faster_rcnn_inception_v2_coco_2018_01_28'
detection_model = load_model(model_name)

In [9]:
print(detection_model.inputs)
print(detection_model.output_dtypes)
print(detection_model.output_shapes)

[<tf.Tensor 'image_tensor:0' shape=(None, None, None, 3) dtype=uint8>]
{'detection_scores': tf.float32, 'detection_classes': tf.float32, 'num_detections': tf.float32, 'detection_boxes': tf.float32}
{'detection_scores': TensorShape([None, 100]), 'detection_classes': TensorShape([None, 100]), 'num_detections': TensorShape([None]), 'detection_boxes': TensorShape([None, 100, 4])}


In [6]:
def run_inference_for_single_image(model, image):
    image = np.asarray(image)
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis,...]

    # Run inference
    output_dict = model(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections

    # detection_classes should be ints.
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

    # Handle models with masks:
    if 'detection_masks' in output_dict:
        # Reframe the the bbox mask to the image size.
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                  output_dict['detection_masks'], output_dict['detection_boxes'],
                   image.shape[0], image.shape[1])      
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                           tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()

    return output_dict

In [8]:
inf_start = time.time()
image_np = np.array(Image.open(".\\images\\2_persons.jpeg"))
# Actual detection.
output_dict = run_inference_for_single_image(detection_model, image_np)
det_time = time.time() - inf_start
print("Inference time: {:.3f}ms".format(det_time * 1000))
print(output_dict['detection_boxes'])
print(output_dict['detection_classes'])
print(output_dict['detection_scores'])
print(output_dict['num_detections'])

Inference time: 491.534ms
[[0.11257906 0.4373718  0.898369   0.61835414]
 [0.14406829 0.65387005 0.96122664 0.7841863 ]
 [0.30732515 0.6562997  0.58599615 0.6883811 ]]
[ 1  1 32]
[0.999796   0.99945325 0.5224417 ]
3
